In [1]:
import os
import shutil
import cv2
import math
import random
import numpy as np
import datetime as dt
import tensorflow
import keras
from collections import deque
import matplotlib.pyplot as plt
plt.style.use("seaborn")

%matplotlib inline

from keras.layers import *
from keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model

C:\Users\teck1\AppData\Local\Temp\ipykernel_26384\2821860772.py:12: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use("seaborn")


In [5]:
from sklearn.model_selection import train_test_split

In [3]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [4]:
import tensorflow as tf
tf.test.is_built_with_cuda()

False

In [5]:
fightdir = "C:/Projects/GitHub/ai/artifacts/data_ingestion/firstdata/fight"
nonfightdir = "C:/Projects/GitHub/ai/artifacts/data_ingestion/firstdata/nofight"
#retrieve the list of video
NonViolence_files_names_list = os.listdir(nonfightdir)
Violence_files_names_list = os.listdir(fightdir)

#Randomly select a video file from the Classes Directory.
Random_NonViolence_Video = random.choice(NonViolence_files_names_list)
Random_Violence_Video = random.choice(Violence_files_names_list)

In [11]:
imageheight, imagewidth = 64, 64
seq = 16
DATASET = "C:/Projects/GitHub/ai/artifacts/data_ingestion/firstdata"

CLASSES_LIST = ["nofight", "fight"]

In [7]:
def frames_extraction(vid_path):
  frames_list = []

  #read vid file
  video_reader = cv2.VideoCapture(vid_path)

  #get total frames in vid
  video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))

  #calculate interval for frames to be added
  skip_frames_window = max(int(video_frames_count/seq), 1)

  for frame_counter in range(seq):
    video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)

    success, frame = video_reader.read()

    if not success:
      break

    #resize
    resized_frame = cv2.resize(frame, (imageheight, imagewidth))
    normalize_frame = resized_frame / 255

    frames_list.append(normalize_frame)

  video_reader.release()

  return frames_list

In [8]:
def create_dataset():
    features = []
    labels = []
    video_files_paths = []
    
    for class_index, class_name in enumerate(CLASSES_LIST):
        files_list = os.listdir(os.path.join(DATASET, class_name))
        
        for file_name in files_list:
            #get directory of the class
            video_file_path = os.path.join(DATASET, class_name, file_name)
            # Extract the frames of the video file.
            frames = frames_extraction(video_file_path)
            if len(frames) == seq:
                # Append the data to their repective lists.
                features.append(frames)
                labels.append(class_index)
                video_files_paths.append(video_file_path)
                
    features = np.asarray(features)
    labels = np.array(labels)  

    return features, labels, video_files_paths

In [12]:
# Create the dataset.
features, labels, video_files_paths = create_dataset()

NameError: name 'create_dataset' is not defined

In [11]:
# Saving the extracted data
np.save("features.npy",features)
np.save("labels.npy",labels)
np.save("video_files_paths.npy",video_files_paths)

In [2]:
features, labels, video_files_paths = np.load("../features.npy") , np.load("../labels.npy") ,  np.load("../video_files_paths.npy")

# # Split and Train Data

In [3]:
# convert labels into one-hot-encoded vectors
one_hot_encoded_labels = to_categorical(labels)

In [6]:
# Split the Data into Train ( 90% ) and Test Set ( 10% ).
features_train, features_test, labels_train, labels_test = train_test_split(features, one_hot_encoded_labels, test_size = 0.1, shuffle = True, random_state = 42)

In [6]:
print(features_train.shape,labels_train.shape)
print(features_test.shape, labels_test.shape)

(1800, 16, 64, 64, 3) (1800, 2)
(200, 16, 64, 64, 3) (200, 2)


In [25]:
from keras.applications.mobilenet_v2 import MobileNetV2

mobilenet = MobileNetV2( include_top=False , weights="imagenet")

#Fine-Tuning to make the last 20 layer trainable
mobilenet.trainable=True

for layer in mobilenet.layers[:-40]:
  layer.trainable=False

#mobilenet.summary()

In [26]:
def create_model():
 
    model = Sequential()

    ########################################################################################################################
    
    #Specifying Input to match features shape
    model.add(Input(shape = (seq, imageheight, imagewidth, 3)))
    
    # Passing mobilenet in the TimeDistributed layer to handle the sequence
    model.add(TimeDistributed(mobilenet))
    
    model.add(Dropout(0.5))
                                    
    model.add(TimeDistributed(Flatten()))

    
    lstm_fw = LSTM(units=32)
    lstm_bw = LSTM(units=32, go_backwards = True)  

    model.add(Bidirectional(lstm_fw, backward_layer = lstm_bw))
    
    # model.add(Dropout(0.25))

    # model.add(Dense(256,activation='relu'))
    # model.add(Dropout(0.25))

    # model.add(Dense(128,activation='relu'))
    # model.add(Dropout(0.25))

    # model.add(Dense(64,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(32,activation='relu'))
    model.add(Dropout(0.5))
    
    
    model.add(Dense(len(CLASSES_LIST), activation = 'softmax'))
 
    ########################################################################################################################
 
    model.summary()
    
    return model

In [27]:
# Constructing the Model
MoBiLSTM_model = create_model()

# Plot the structure of the contructed LRCN model.
plot_model(MoBiLSTM_model, to_file = 'MobBiLSTM_model_structure_plot.png', show_shapes = True, show_layer_names = True)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_6 (TimeDi  (None, 16, 2, 2, 1280)    2257984   
 stributed)                                                      
                                                                 
 dropout_15 (Dropout)        (None, 16, 2, 2, 1280)    0         
                                                                 
 time_distributed_7 (TimeDi  (None, 16, 5120)          0         
 stributed)                                                      
                                                                 
 bidirectional_3 (Bidirecti  (None, 64)                1319168   
 onal)                                                           
                                                                 
 dropout_16 (Dropout)        (None, 64)                0         
                                                      

In [28]:
# Create Early Stopping Callback to monitor the accuracy
early_stopping_callback = EarlyStopping(monitor = 'val_accuracy', patience = 10, restore_best_weights = True)

# Create ReduceLROnPlateau Callback to reduce overfitting by decreasing learning
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                  factor=0.6,
                                                  patience=5,
                                                  min_lr=0.00001,
                                                  verbose=1)
 
# Compiling the model 
MoBiLSTM_model.compile(loss = 'categorical_crossentropy', optimizer = 'sgd', metrics = ["accuracy"])
 
# Fitting the model 
MobBiLSTM_model_history = MoBiLSTM_model.fit(x = features_train, y = labels_train, epochs = 80, batch_size = 8 ,
                                             shuffle = True, validation_split = 0.2, callbacks = [early_stopping_callback,reduce_lr])

Epoch 1/80
180/180 [==============================] - 55s 269ms/step - loss: 0.7628 - accuracy: 0.4688 - val_loss: 0.6911 - val_accuracy: 0.5028 - lr: 0.0100
Epoch 2/80
180/180 [==============================] - 45s 251ms/step - loss: 0.7050 - accuracy: 0.5236 - val_loss: 0.6868 - val_accuracy: 0.5917 - lr: 0.0100
Epoch 3/80
180/180 [==============================] - 45s 251ms/step - loss: 0.7093 - accuracy: 0.5153 - val_loss: 0.6888 - val_accuracy: 0.5944 - lr: 0.0100
Epoch 4/80
180/180 [==============================] - 46s 255ms/step - loss: 0.7072 - accuracy: 0.5021 - val_loss: 0.6926 - val_accuracy: 0.5222 - lr: 0.0100
Epoch 5/80
180/180 [==============================] - 46s 256ms/step - loss: 0.6907 - accuracy: 0.5236 - val_loss: 0.6844 - val_accuracy: 0.6111 - lr: 0.0100
Epoch 6/80
180/180 [==============================] - 46s 255ms/step - loss: 0.6876 - accuracy: 0.5361 - val_loss: 0.6774 - val_accuracy: 0.6472 - lr: 0.0100
Epoch 7/80
180/180 [==============================] 

In [29]:
model_evaluation_history = MoBiLSTM_model.evaluate(features_test, labels_test)

7/7 [==============================] - 4s 294ms/step - loss: 0.6971 - accuracy: 0.7150


In [31]:
MoBiLSTM_model.save('artifacts/training/model_v2.h5', include_optimizer=True)

: 